In [8]:
from sklearn.metrics import accuracy_score, roc_auc_score
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision import transforms
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from tqdm import tqdm
import numpy as np
import torch
import json

In [9]:
def one_hot_labels(labels):
    r = np.zeros((labels.size, labels.max()+1))
    r[np.arange(labels.size),labels] = 1
    return r

In [11]:
import SIFT.SIFT_classes as SIFT_classes
path="Best_Models/SIFTRetinaStart/weights.ckpt"
model = SIFT_classes.SIFTRetinaStart.load_from_checkpoint(path)

RuntimeError: Error(s) in loading state_dict for SIFTRetinaStart:
	size mismatch for vvs_fc.weight: copying a param with shape torch.Size([1966, 98304]) from checkpoint, the shape in current model is torch.Size([768, 98304]).
	size mismatch for vvs_fc.bias: copying a param with shape torch.Size([1966]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for outputs.weight: copying a param with shape torch.Size([10, 1966]) from checkpoint, the shape in current model is torch.Size([10, 768]).

In [ ]:
# Get test dataloader
transform = transforms.Compose([
        transforms.Grayscale(1),
        transforms.ToTensor()
    ])
test_data = CIFAR10(root="data/CIFAR10", download=False, train=False, transform=transform)
test_loader = DataLoader(test_data, batch_size=32, num_workers=12)
print ("DataLoader acquired")

In [ ]:
# Prediction
y_true = torch.cat([labels for _, labels in test_loader]).reshape(-1).numpy()
y_roc = one_hot_labels(y_true)
with torch.no_grad(): y_pred = torch.cat([model(images) for images, _ in test_loader]).detach().numpy()
print ("Predictions calculated")

In [ ]:
auc = roc_auc_score(y_roc, y_pred, multi_class="ovr")
print("Model AUC:", auc)

In [ ]:
model_acc = accuracy_score(y_true, y_pred.argmax(-1))
print("Model accuracy:", model_acc)